## Answer 1




In [2]:
import numpy as np
import pandas as pd

def construct_diversified_portfolio(assets_returns):
    """
    Constructs a diversified portfolio by selecting assets with the lowest correlation.

    Parameters:
    - assets_returns: a dictionary where keys are asset names and values are pandas Series of daily returns.

    Returns:
    - list of asset names selected for the diversified portfolio.
    """

    # 1:  daily returns to a DataFrame
    returns_df = pd.DataFrame(assets_returns)

    # 2: correlation matrix of daily returns
    correlation_matrix = returns_df.corr()

    # 3: Select a subset of assets that are least correlated to each other
    min_avg_corr = np.inf
    best_assets = []

    # Iterate through assets
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            asset_i = correlation_matrix.columns[i]
            asset_j = correlation_matrix.columns[j]
            avg_corr = (correlation_matrix[asset_i][asset_j] + correlation_matrix[asset_j][asset_i]) / 2.0

            if avg_corr < min_avg_corr:
                min_avg_corr = avg_corr
                best_assets = [asset_i, asset_j]

    
    return best_assets


In [3]:
# Example usage:
if __name__ == "__main__":
    # Example daily returns data (replace with your actual data)
    asset1_returns = pd.Series([0.01, 0.02, -0.01, 0.005, 0.015])
    asset2_returns = pd.Series([0.015, -0.01, 0.02, 0.025, -0.005])
    asset3_returns = pd.Series([-0.005, 0.01, 0.015, 0.005, -0.01])

    assets_returns = {
        'Asset1': asset1_returns,
        'Asset2': asset2_returns,
        'Asset3': asset3_returns
    }

    selected_assets = construct_diversified_portfolio(assets_returns)
    print("Selected subset of assets:", selected_assets)

Selected subset of assets: ['Asset1', 'Asset2']


# Answer 2


In [13]:
import yfinance as yf
import pandas as pd
import numpy as np

def fetch_historical_data(tickers, start_date, end_date):

    data = yf.download(tickers, start=start_date, end=end_date)
    return data['Adj Close']  # Extract adjusted closing prices

def calculate_returns(prices):
    
    #Calculate daily returns from historical prices.
    return prices.pct_change().dropna()

def calculate_portfolio_returns(returns, weights):
    """
    Calculate portfolio returns based on given asset returns and weights.
    
    Parameters:
    - returns: pandas DataFrame containing daily returns of assets.
    - weights: Dictionary where keys are asset names and values are weights (summing up to 1).
    
    """
    portfolio_returns = returns.dot(pd.Series(weights))
    return portfolio_returns

def calculate_var(portfolio_returns, confidence_level=0.05):
    """
    Parameters:
    - portfolio_returns: pandas Series containing portfolio returns.
    - confidence_level: Confidence level for VaR calculation, default is 0.05 (95% confidence).   .
    """
    return np.percentile(portfolio_returns, confidence_level * 100)

def save_to_csv(data, file_path):    
    data.to_csv(file_path)




In [12]:

if __name__ == "__main__":
    tickers = ['AAPL', 'MSFT', 'GOOGL']  # Example t
    start_date = '2023-01-01'
    end_date = '2023-12-31'
    file_path = 'historical_data.csv'  #output file

    # 1:fetching of file
    data = fetch_historical_data(tickers, start_date, end_date)

    # 2: daily returns
    returns = calculate_returns(data)

    # 3:  portfolio weights (example weights)
    weights = {'AAPL': 0.4, 'MSFT': 0.3, 'GOOGL': 0.3}

    # 4: portfolio returns
    portfolio_returns = calculate_portfolio_returns(returns, weights)

    # 5: VaR at 95% confidence level
    var_95 = calculate_var(portfolio_returns, confidence_level=0.05)

    # Output 
    print(f"Value at Risk (VaR) at 95% confidence level: {var_95:.2%}")

    # 6: Save data to CSV
    save_to_csv(data, file_path)

    print(f"Data saved to {file_path}")

[*********************100%%**********************]  3 of 3 completed

Value at Risk (VaR) at 95% confidence level: -1.92%
Data saved to historical_data.csv
